In [1]:
import os
#import jovian
import matplotlib
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [2]:
df_boxes = pd.read_csv('boxes.csv') #importing box details datasets
df_problem1 = pd.read_csv('problem 1.csv') #importing predict datasets
df_purchase = pd.read_csv('purchase.csv') #importing history datasets

## Data Cleaning

In [3]:
nan_counts = df_purchase.isnull().sum()
print(nan_counts)

PURCHASE_DATE     0
MAGIC_KEY         0
BOX_ID           47
BOX_COUNT        47
dtype: int64


In [4]:
BOX_ID_Mode = df_purchase['BOX_ID'].mode()[0]
mean_box_count = df_purchase['BOX_COUNT'].mean()

# Fill NaN values with mean values
df_purchase['BOX_ID'].fillna(BOX_ID_Mode, inplace=True)
df_purchase['BOX_COUNT'].fillna(mean_box_count, inplace=True)

In [5]:
nan_counts = df_purchase.isnull().sum()
print(nan_counts)

PURCHASE_DATE    0
MAGIC_KEY        0
BOX_ID           0
BOX_COUNT        0
dtype: int64


In [6]:
# Fill missing values for the magic ID '290D33249B7' with previous data
df_purchase.loc[df_purchase['MAGIC_KEY'] == '290D33249B7', ['BOX_ID', 'BOX_COUNT']] = df_purchase.loc[df_purchase['MAGIC_KEY'] == '290D33249B7', ['BOX_ID', 'BOX_COUNT']].fillna(method='ffill')

C:\Users\USER\AppData\Local\Temp\ipykernel_1184\1714574754.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_purchase.loc[df_purchase['MAGIC_KEY'] == '290D33249B7', ['BOX_ID', 'BOX_COUNT']] = df_purchase.loc[df_purchase['MAGIC_KEY'] == '290D33249B7', ['BOX_ID', 'BOX_COUNT']].fillna(method='ffill')


In [7]:
merged_df = pd.merge(df_boxes, df_purchase, on='BOX_ID')

In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2455847 entries, 0 to 2455846
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   BOX_ID           int64  
 1   QUALITY          object 
 2   DELIVERY_OPTION  object 
 3   MILK             float64
 4   MEAT             float64
 5   UNIT_PRICE       float64
 6   PURCHASE_DATE    object 
 7   MAGIC_KEY        object 
 8   BOX_COUNT        float64
dtypes: float64(4), int64(1), object(4)
memory usage: 168.6+ MB


In [9]:
total_unique_box_id = merged_df['BOX_ID'].nunique()

print("Total unique BOX_ID:", total_unique_box_id)

Total unique BOX_ID: 290


In [10]:
total_unique_magic_key = merged_df['MAGIC_KEY'].nunique()

print("Total unique BOX_ID:", total_unique_magic_key)

Total unique BOX_ID: 1274101


## data cleaning

In [11]:
round(df_purchase.describe())

,BOX_ID,BOX_COUNT
count,2455864.0,2455864.0
mean,231.0,1.0
std,29233.0,0.0
min,1.0,-1.0
25%,106.0,1.0
50%,143.0,1.0
75%,215.0,1.0
max,11111111.0,19.0


In [12]:
round(merged_df.describe())

,BOX_ID,MILK,MEAT,UNIT_PRICE,BOX_COUNT
count,2455847.0,2455847.0,2455847.0,2455847.0,2455847.0
mean,154.0,8.0,2.0,18.0,1.0
std,70.0,6.0,1.0,4.0,0.0
min,1.0,0.0,0.0,6.0,1.0
25%,106.0,0.0,2.0,14.0,1.0
50%,143.0,10.0,2.0,18.0,1.0
75%,215.0,11.0,2.0,20.0,1.0
max,290.0,24.0,6.0,24.0,19.0


In [13]:
merged_df.duplicated().sum()

77

In [14]:
# drop duplicates rows
merged_df= merged_df.drop_duplicates()

In [15]:
from scipy import stats

z_scores = np.abs(stats.zscore(df_purchase['BOX_COUNT']))
threshold = 3
outliers = np.where(z_scores > threshold)[0]
print("Outliers detected using Z-score method:", outliers)

In [16]:
merged_df.head()

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,4/2/2019,2C88D36D1FC,1.0
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2BF011BDB38,1.0
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA0EE8F2B3,1.0
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2C623730B09,1.0
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA6CE1054F,1.0


In [17]:
merged_df['PURCHASE_DATE'] = pd.to_datetime(merged_df['PURCHASE_DATE'], format="%d/%m/%Y")

In [18]:
#merged_df.PURCHASE_DATE.min(), merged_df.PURCHASE_DATE.max()

In [19]:
merged_df.PURCHASE_DATE.info()

<class 'pandas.core.series.Series'>
Index: 2455770 entries, 0 to 2455846
Series name: PURCHASE_DATE
Non-Null Count    Dtype         
--------------    -----         
2455770 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 37.5 MB


# feature engineering

### Adding the target column

In [20]:
merged_df['PURCHASE'] = 'Y'

In [21]:
merged_df.loc[merged_df['PURCHASE_DATE'].dt.day > 15, 'PURCHASE'] = 'N'

In [22]:
# Group by 'MAGIC_KEY' and count the number of purchases for each customer
temp_df = merged_df.groupby('MAGIC_KEY')['PURCHASE'].count().reset_index()
temp_df.columns = ['MAGIC_KEY', 'PURCHASE_COUNT']


In [23]:
# Calculate the total amount spent for each purchase
merged_df['TOTAL_AMOUNT'] = merged_df['BOX_COUNT'] * merged_df['UNIT_PRICE']

# Group by 'MAGIC_KEY' and sum the total amount spent by each customer
total_amount_spent = merged_df.groupby('MAGIC_KEY')['TOTAL_AMOUNT'].sum().reset_index()
total_amount_spent.columns = ['MAGIC_KEY', 'TOTAL_AMOUNT_SPENT']


In [24]:
df_customer = pd.merge(temp_df, total_amount_spent, on='MAGIC_KEY')
df_customer

,MAGIC_KEY,PURCHASE_COUNT,TOTAL_AMOUNT_SPENT
0,249670911D8,2,24.10
1,249751FC4DD,1,17.98
2,24978027606,1,15.96
3,24979164422,1,13.96
4,2497B8B4FDA,2,33.94
...,...,...,...
1274096,2E6F72C6F1C,3,51.94
1274097,2E6F8194908,1,11.96
1274098,2E6F9C7B9B4,1,19.98
1274099,2E6FB0EBB32,6,107.88


In [25]:
merged_df = pd.merge(df_customer, merged_df, on='MAGIC_KEY')

In [26]:

#merged_df

In [27]:
last_purchase_date = merged_df.groupby('MAGIC_KEY')['PURCHASE_DATE'].max().reset_index()
last_purchase_date.columns = ['MAGIC_KEY', 'LAST_PURCHASE_DATE']

# Display the DataFrame with the last purchase date for each customer
last_purchase_date

,MAGIC_KEY,LAST_PURCHASE_DATE
0,249670911D8,2019-02-02
1,249751FC4DD,2018-10-12
2,24978027606,2018-10-19
3,24979164422,2018-11-30
4,2497B8B4FDA,2018-11-01
...,...,...
1274096,2E6F72C6F1C,2019-02-27
1274097,2E6F8194908,2019-02-23
1274098,2E6F9C7B9B4,2019-01-28
1274099,2E6FB0EBB32,2019-02-11


In [28]:
last_purchase_date['DAY_SINCE_LAST_PURCHASE'] = pd.to_datetime('2019-02-01') - last_purchase_date['LAST_PURCHASE_DATE']
last_purchase_date

,MAGIC_KEY,LAST_PURCHASE_DATE,DAY_SINCE_LAST_PURCHASE
0,249670911D8,2019-02-02,-1 days
1,249751FC4DD,2018-10-12,112 days
2,24978027606,2018-10-19,105 days
3,24979164422,2018-11-30,63 days
4,2497B8B4FDA,2018-11-01,92 days
...,...,...,...
1274096,2E6F72C6F1C,2019-02-27,-26 days
1274097,2E6F8194908,2019-02-23,-22 days
1274098,2E6F9C7B9B4,2019-01-28,4 days
1274099,2E6FB0EBB32,2019-02-11,-10 days


In [29]:
merged_df = pd.merge(merged_df, last_purchase_date, on='MAGIC_KEY')


In [30]:
# Calculate the time difference between consecutive purchases for each customer
merged_df['TIME_DIFF'] = merged_df.groupby('MAGIC_KEY')['PURCHASE_DATE'].diff()

In [31]:
# Group by 'MAGIC_KEY' and calculate the average time difference between consecutive purchases
average_diff = merged_df.groupby('MAGIC_KEY')['TIME_DIFF'].mean().reset_index()
average_diff.columns = ['MAGIC_KEY', 'AVERAGE_DIFF']

# Fill NaN values (customers with only one purchase) with 0
average_diff['AVERAGE_DIFF'] = average_diff['AVERAGE_DIFF'].fillna(0)
average_diff['AVERAGE_DIFF'] = pd.to_timedelta(average_diff['AVERAGE_DIFF'])

In [32]:
merged_df = pd.merge(merged_df, average_diff, on='MAGIC_KEY')


In [33]:
#---------------------------------------------------OCOTBER------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in October 2018
october_2018_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2018) & 
                             (merged_df['PURCHASE_DATE'].dt.month == 10)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of October 2018
october_2018_first_15_days_df = october_2018_df[october_2018_df['PURCHASE_DATE'].dt.day <= 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
october_2018_first_15_days_purchases = october_2018_first_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "10_FIRST_PURCHASE_2018" based on whether a purchase is made by each Magic Key
merged_df['10_FIRST_PURCHASE_2018'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(october_2018_first_15_days_purchases.index), '10_FIRST_PURCHASE_2018'] = 1


#---------------------------------------------------------------------------------------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in October 2018
october_2018_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2018) & 
                             (merged_df['PURCHASE_DATE'].dt.month == 10)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of October 2018
october_2018_last_15_days_df = october_2018_df[october_2018_df['PURCHASE_DATE'].dt.day > 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
october_2018_last_15_days_purchases = october_2018_last_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "10_FIRST_PURCHASE_2018" based on whether a purchase is made by each Magic Key
merged_df['10_LAST_PURCHASE_2018'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(october_2018_last_15_days_purchases.index), '10_LAST_PURCHASE_2018'] = 1


#_____________________________________________________________ NOVEMBER------------------------------

# Filter the DataFrame to include only the rows with purchase dates in November 2018
november_2018_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2018) & 
                             (merged_df['PURCHASE_DATE'].dt.month == 11)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of November 2018
november_2018_first_15_days_df = november_2018_df[november_2018_df['PURCHASE_DATE'].dt.day <= 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
november_2018_first_15_days_purchases = november_2018_first_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "11_FIRST_PURCHASE_2018" based on whether a purchase is made by each Magic Key
merged_df['11_FIRST_PURCHASE_2018'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(november_2018_first_15_days_purchases.index), '11_FIRST_PURCHASE_2018'] = 1


#---------------------------------------------------------------------------------------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in November 2018
november_2018_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2018) & 
                             (merged_df['PURCHASE_DATE'].dt.month == 11)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of November 2018
november_2018_first_15_days_df = november_2018_df[november_2018_df['PURCHASE_DATE'].dt.day > 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
november_2018_last_15_days_purchases = november_2018_first_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "11_FIRST_PURCHASE_2018" based on whether a purchase is made by each Magic Key
merged_df['11_LAST_PURCHASE_2018'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(november_2018_first_15_days_purchases.index), '11_LAST_PURCHASE_2018'] = 1


#------------------------------------------------------- DECEMBER---------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in December 2018
december_2018_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2018) & 
                             (merged_df['PURCHASE_DATE'].dt.month == 11)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of december 2018
december_2018_first_15_days_df = december_2018_df[december_2018_df['PURCHASE_DATE'].dt.day <= 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
december_2018_first_15_days_purchases = december_2018_first_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "11_FIRST_PURCHASE_2018" based on whether a purchase is made by each Magic Key
merged_df['12_FIRST_PURCHASE_2018'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(december_2018_first_15_days_purchases.index), '12_FIRST_PURCHASE_2018'] = 1

#---------------------------------------------------------------------------------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in December 2018
december_2018_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2018) & 
                             (merged_df['PURCHASE_DATE'].dt.month == 11)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of december 2018
december_2018_last_15_days_df = december_2018_df[december_2018_df['PURCHASE_DATE'].dt.day > 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
december_2018_last_15_days_purchases = december_2018_last_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "11_FIRST_PURCHASE_2018" based on whether a purchase is made by each Magic Key
merged_df['12_LAST_PURCHASE_2018'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(december_2018_last_15_days_purchases.index), '12_LAST_PURCHASE_2018'] = 1



#---------------------------------------------------JANUARY 2019------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in January 2019
january_2019_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2019) & 
                          (merged_df['PURCHASE_DATE'].dt.month == 1)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of January 2019
january_2019_first_15_days_df = january_2019_df[january_2019_df['PURCHASE_DATE'].dt.day <= 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
january_2019_first_15_days_purchases = january_2019_first_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "01_FIRST_PURCHASE_2019" based on whether a purchase is made by each Magic Key
merged_df['01_FIRST_PURCHASE_2019'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(january_2019_first_15_days_purchases.index), '01_FIRST_PURCHASE_2019'] = 1


#---------------------------------------------------------------------------------------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in January 2019
january_2019_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2019) & 
                          (merged_df['PURCHASE_DATE'].dt.month == 1)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of January 2019
january_2019_last_15_days_df = january_2019_df[january_2019_df['PURCHASE_DATE'].dt.day > 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
january_2019_last_15_days_purchases = january_2019_last_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "01_LAST_PURCHASE_2019" based on whether a purchase is made by each Magic Key
merged_df['01_LAST_PURCHASE_2019'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(january_2019_last_15_days_purchases.index), '01_LAST_PURCHASE_2019'] = 1


#_____________________________________________________________ FEBRUARY 2019----------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in February 2019
february_2019_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2019) & 
                            (merged_df['PURCHASE_DATE'].dt.month == 2)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of February 2019
february_2019_first_15_days_df = february_2019_df[february_2019_df['PURCHASE_DATE'].dt.day <= 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
february_2019_first_15_days_purchases = february_2019_first_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "02_FIRST_PURCHASE_2019" based on whether a purchase is made by each Magic Key
merged_df['02_FIRST_PURCHASE_2019'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(february_2019_first_15_days_purchases.index), '02_FIRST_PURCHASE_2019'] = 1

#---------------------------------------------------------------------------------------------------------------------

# Filter the DataFrame to include only the rows with purchase dates in February 2019
february_2019_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == 2019) & 
                            (merged_df['PURCHASE_DATE'].dt.month == 2)]

# Further filter the DataFrame to include only the rows with purchase dates within the first 15 days of February 2019
february_2019_last_15_days_df = february_2019_df[february_2019_df['PURCHASE_DATE'].dt.day > 15]

# Group the filtered DataFrame by "MAGIC_KEY" and check if any purchase of MILK or MEAT is made by each Magic Key
february_2019_last_15_days_purchases = february_2019_last_15_days_df.groupby('MAGIC_KEY')[['MILK', 'MEAT']].sum()

# Create the new column "02_FIRST_PURCHASE_2019" based on whether a purchase is made by each Magic Key
merged_df['02_LAST_PURCHASE_2019'] = 0
merged_df.loc[merged_df['MAGIC_KEY'].isin(february_2019_last_15_days_purchases.index), '02_LAST_PURCHASE_2019'] = 1


In [34]:
# Group by 'MAGIC_KEY' and count the number of purchases for each customer
df_customer = merged_df.groupby('MAGIC_KEY')['PURCHASE'].count().reset_index()
df_customer.columns = ['MAGIC_KEY', 'PURCHASE_COUNT']


In [35]:
merged_df = pd.merge(df_customer, merged_df, on='MAGIC_KEY')


In [36]:
merged_df['DAY_SINCE_LAST_PURCHASE'] = pd.to_datetime('2019-03-01') - last_purchase_date['LAST_PURCHASE_DATE']

In [37]:
# Group the data by MAGIC_KEY and calculate the sum of MEAT purchased by each Magic Key
magic_key_meat = merged_df.groupby('MAGIC_KEY')['MEAT'].sum().reset_index(name= 'TOTAL_MEAT')

# Count the frequency of purchases made by each Magic Key
magic_key_purchase_frequency = merged_df.groupby('MAGIC_KEY').size().reset_index(name='MEAT_PURCHASE_FREEQUENCY')

# Merge the two DataFrames on MAGIC_KEY
magic_key_data = pd.merge(magic_key_meat, magic_key_purchase_frequency, on='MAGIC_KEY')

# Display the resulting DataFrame
#print(magic_key_data)

In [38]:
# Assuming MAGIC_KEY is the common column between merged_df and magic_key_data
merged_df = pd.merge(merged_df, magic_key_data, on='MAGIC_KEY')


In [39]:
# Group the data by MAGIC_KEY and calculate the sum of MEAT purchased by each Magic Key
magic_key_milk = merged_df.groupby('MAGIC_KEY')['MILK'].sum().reset_index(name= 'TOTAL_MILK')

# Count the frequency of purchases made by each Magic Key
magic_key_purchase_frequency_1 = merged_df.groupby('MAGIC_KEY').size().reset_index(name='MILK_PURCHASE_FREEQUENCY')

# Merge the two DataFrames on MAGIC_KEY
magic_key_data_1 = pd.merge(magic_key_meat, magic_key_purchase_frequency_1, on='MAGIC_KEY')


In [40]:
# Assuming MAGIC_KEY is the common column between merged_df and magic_key_data
merged_df = pd.merge(merged_df, magic_key_data_1, on='MAGIC_KEY')


In [41]:
merged_df

,MAGIC_KEY,PURCHASE_COUNT_x,PURCHASE_COUNT_y,TOTAL_AMOUNT_SPENT,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,...,12_FIRST_PURCHASE_2018,12_LAST_PURCHASE_2018,01_FIRST_PURCHASE_2019,01_LAST_PURCHASE_2019,02_FIRST_PURCHASE_2019,02_LAST_PURCHASE_2019,TOTAL_MEAT_x,MEAT_PURCHASE_FREEQUENCY,TOTAL_MEAT_y,MILK_PURCHASE_FREEQUENCY
0,249670911D8,2,2,24.10,231,Standard,Delivery from Collection Point,8.5,0.0,10.14,...,1,0,0,0,1,0,2.4,2,2.4,2
1,249670911D8,2,2,24.10,245,Standard,Delivery from Collection Point,0.0,2.4,13.96,...,1,0,0,0,1,0,2.4,2,2.4,2
2,249751FC4DD,1,1,17.98,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,...,0,0,0,0,0,0,1.8,1,1.8,1
3,24978027606,1,1,15.96,27,Premium,Home Delivery - CoD,0.0,2.9,15.96,...,0,0,0,0,0,0,2.9,1,2.9,1
4,24979164422,1,1,13.96,246,Standard,Delivery from Collection Point,0.0,2.5,13.96,...,0,1,0,0,0,0,2.5,1,2.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455765,2E6FB0EBB32,6,6,107.88,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,...,0,1,0,1,1,0,12.8,6,12.8,6
2455766,2E6FB0EBB32,6,6,107.88,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,...,0,1,0,1,1,0,12.8,6,12.8,6
2455767,2E6FB0EBB32,6,6,107.88,258,Standard,Delivery from Collection Point,8.0,2.2,17.98,...,0,1,0,1,1,0,12.8,6,12.8,6
2455768,2E6FB0EBB32,6,6,107.88,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,...,0,1,0,1,1,0,12.8,6,12.8,6


## Let's also parse the date column

## extracting Day, Month, Year from the dataset and making new columns of them

In [42]:
merged_df['DAY'] = merged_df['PURCHASE_DATE'].dt.day
merged_df['MONTH'] = merged_df['PURCHASE_DATE'].dt.month
merged_df['YEAR'] = merged_df['PURCHASE_DATE'].dt.year

## adding month wise purchase 

In [43]:
## this is the cols of first 15 dayas of meat purchase  and last 15 days of meat purchase from oct to feb




# # Function to create purchase columns for a specific month and year
# def create_purchase_columns(df, month, year):
#     # Filter the DataFrame to include only the rows with purchase dates in the specified month and year
#     filtered_df = merged_df[(merged_df['PURCHASE_DATE'].dt.year == year) & 
#                             (merged_df['PURCHASE_DATE'].dt.month == month)]
    
#     # Further filter the DataFrame to include only the rows with purchase dates within the first 15 days or last 15 days of the month
#     first_15_days_df = filtered_df[filtered_df['PURCHASE_DATE'].dt.day <= 15]
#     last_15_days_df = filtered_df[filtered_df['PURCHASE_DATE'].dt.day > 15]
    
#     # Group the filtered DataFrame by "MAGIC_KEY" and sum the "MEAT" column for each group
#     first_15_days_purchases = first_15_days_df.groupby('MAGIC_KEY')['MEAT'].sum()
#     last_15_days_purchases = last_15_days_df.groupby('MAGIC_KEY')['MEAT'].sum()
    
#     # Create new columns based on the total meat purchased by each Magic Key in the specified time periods
#     col_first_purchase = f"{month:02d}_FIRST_PURCHASE_{year}"
#     col_last_purchase = f"{month:02d}_LAST_PURCHASE_{year}"
    
#     merged_df[col_first_purchase] = 0
#     merged_df.loc[merged_df['MAGIC_KEY'].isin(first_15_days_purchases.index), col_first_purchase] = first_15_days_purchases
    
#     merged_df[col_last_purchase] = 0
#     merged_df.loc[merged_df['MAGIC_KEY'].isin(last_15_days_purchases.index), col_last_purchase] = last_15_days_purchases

# # Create purchase columns for each month and year from November 2018 to February 2019
# for month, year in [(10, 2018),(11, 2018), (12, 2018), (1, 2019), (2, 2019)]:
#     create_purchase_columns(merged_df, month, year)


### Month WIse meat purchase

In [44]:
# Create a pivot table to calculate the total meat purchased by each magic key in each month
monthly_meat_purchases = merged_df.pivot_table(index='MAGIC_KEY', columns=merged_df['PURCHASE_DATE'].dt.month, values='MEAT', aggfunc='sum')

# Rename the columns to represent the respective months
monthly_meat_purchases.columns = ['OCT_MEAT', 'NOV_MEAT', 'DEC_MEAT', 'JAN_MEAT', 'FEB_MEAT']

# Reset the index to convert the MAGIC_KEY back to a column
monthly_meat_purchases.reset_index(inplace=True)

# Fill NaN values with 0
monthly_meat_purchases.fillna(0, inplace=True)

# Display the resulting DataFrame
#print(monthly_meat_purchases)


In [45]:
merged_df = pd.merge(merged_df, monthly_meat_purchases, on='MAGIC_KEY')

### Month WIse milK purchase

In [46]:
# Create a pivot table to calculate the total milk purchased by each magic key in each month
monthly_milk_purchases = merged_df.pivot_table(index='MAGIC_KEY', columns=merged_df['PURCHASE_DATE'].dt.month, values='MILK', aggfunc='sum')

# Rename the columns to represent the respective months
monthly_milk_purchases.columns = ['OCT_MILK', 'NOV_MILK', 'DEC_MILK', 'JAN_MILK', 'FEB_MILK']

# Reset the index to convert the MAGIC_KEY back to a column
monthly_milk_purchases.reset_index(inplace=True)

# Fill NaN values with 0
monthly_milk_purchases.fillna(0, inplace=True)

In [47]:
merged_df = pd.merge(merged_df, monthly_milk_purchases, on='MAGIC_KEY')

In [48]:
#merged_df.PURCHASE_DATE.min(), merged_df.PURCHASE_DATE.max()

In [49]:
#merged_df.BOX_COUNT.value_counts()

In [50]:
#temp_df = merged_df.copy()

In [51]:
# Assuming the Magic Key you want to search for is '290D33249B7'
merged_df.loc[merged_df['MAGIC_KEY'] == '290D33249B7']

# Display the specific row



,MAGIC_KEY,PURCHASE_COUNT_x,PURCHASE_COUNT_y,TOTAL_AMOUNT_SPENT,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,...,OCT_MEAT,NOV_MEAT,DEC_MEAT,JAN_MEAT,FEB_MEAT,OCT_MILK,NOV_MILK,DEC_MILK,JAN_MILK,FEB_MILK
358692,290D33249B7,4,4,96.027488,163,Premium,Delivery from Collection Point,11.0,2.2,23.98,...,0.0,6.6,0.0,0.0,2.2,0.0,33.0,0.0,0.0,11.0
358693,290D33249B7,4,4,96.027488,163,Premium,Delivery from Collection Point,11.0,2.2,23.98,...,0.0,6.6,0.0,0.0,2.2,0.0,33.0,0.0,0.0,11.0
358694,290D33249B7,4,4,96.027488,163,Premium,Delivery from Collection Point,11.0,2.2,23.98,...,0.0,6.6,0.0,0.0,2.2,0.0,33.0,0.0,0.0,11.0
358695,290D33249B7,4,4,96.027488,163,Premium,Delivery from Collection Point,11.0,2.2,23.98,...,0.0,6.6,0.0,0.0,2.2,0.0,33.0,0.0,0.0,11.0


# EDA 

In [52]:
# corr_matrix = merged_df.corr()

# # Plot the heatmap
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Correlation Heatmap')
# plt.show()

In [53]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2455770 entries, 0 to 2455769
Data columns (total 45 columns):
 #   Column                    Dtype          
---  ------                    -----          
 0   MAGIC_KEY                 object         
 1   PURCHASE_COUNT_x          int64          
 2   PURCHASE_COUNT_y          int64          
 3   TOTAL_AMOUNT_SPENT        float64        
 4   BOX_ID                    int64          
 5   QUALITY                   object         
 6   DELIVERY_OPTION           object         
 7   MILK                      float64        
 8   MEAT                      float64        
 9   UNIT_PRICE                float64        
 10  PURCHASE_DATE             datetime64[ns] 
 11  BOX_COUNT                 float64        
 12  PURCHASE                  object         
 13  TOTAL_AMOUNT              float64        
 14  LAST_PURCHASE_DATE        datetime64[ns] 
 15  DAY_SINCE_LAST_PURCHASE   timedelta64[ns]
 16  TIME_DIFF                 timedelta6

## COnverting DATE_TIME datatypes to int

In [54]:
merged_df['LAST_PURCHASE_DATE'] = pd.to_datetime(merged_df['LAST_PURCHASE_DATE'])
merged_df['DAY_SINCE_LAST_PURCHASE'] = pd.to_timedelta(merged_df['DAY_SINCE_LAST_PURCHASE'])
merged_df['AVERAGE_DIFF'] = pd.to_timedelta(merged_df['AVERAGE_DIFF'])


In [55]:
import numpy as np

# Convert timedelta columns to integer after handling non-finite values
merged_df['DAY_SINCE_LAST_PURCHASE'] = merged_df['DAY_SINCE_LAST_PURCHASE'].dt.days.fillna(0).astype(int)
merged_df['AVERAGE_DIFF'] = merged_df['AVERAGE_DIFF'].dt.days.fillna(0).astype(int)


In [56]:
merged_df.drop(columns=['LAST_PURCHASE_DATE'], inplace=True)

In [57]:
merged_df.PURCHASE_DATE.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2455770 entries, 0 to 2455769
Series name: PURCHASE_DATE
Non-Null Count    Dtype         
--------------    -----         
2455770 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 18.7 MB


In [58]:
merged_df.to_csv('merged_data_2.csv', index=False)

In [60]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2455770 entries, 0 to 2455769
Data columns (total 44 columns):
 #   Column                    Dtype          
---  ------                    -----          
 0   MAGIC_KEY                 object         
 1   PURCHASE_COUNT_x          int64          
 2   PURCHASE_COUNT_y          int64          
 3   TOTAL_AMOUNT_SPENT        float64        
 4   BOX_ID                    int64          
 5   QUALITY                   object         
 6   DELIVERY_OPTION           object         
 7   MILK                      float64        
 8   MEAT                      float64        
 9   UNIT_PRICE                float64        
 10  PURCHASE_DATE             datetime64[ns] 
 11  BOX_COUNT                 float64        
 12  PURCHASE                  object         
 13  TOTAL_AMOUNT              float64        
 14  DAY_SINCE_LAST_PURCHASE   int32          
 15  TIME_DIFF                 timedelta64[ns]
 16  AVERAGE_DIFF              int32     